# 作業 : (Kaggle)鐵達尼生存預測
https://www.kaggle.com/c/titanic

# [作業目標]
- 試著模仿範例寫法, 在鐵達尼生存預測中, 觀察填補缺值以及 標準化 / 最小最大化 對數值的影響

# [作業重點]
- 觀察替換不同補缺方式, 對於特徵的影響 (In[4]~In[6], Out[4]~Out[6])
- 觀察替換不同特徵縮放方式, 對於特徵的影響 (In[7]~In[8], Out[7]~Out[8])

In [1]:
# 做完特徵工程前的所有準備 (與前範例相同)
import pandas as pd
import numpy as np
import copy
from sklearn.preprocessing import MinMaxScaler, StandardScaler
from sklearn.model_selection import cross_val_score
from sklearn.linear_model import LogisticRegression

# 忽略警告訊息
%matplotlib inline
import warnings
warnings.filterwarnings('ignore')

data_path = '../data/'
df_train = pd.read_csv(data_path + 'titanic_train.csv')
df_test = pd.read_csv(data_path + 'titanic_test.csv')

train_Y = df_train['Survived']
ids = df_test['PassengerId']
df_train = df_train.drop(['PassengerId', 'Survived'] , axis=1)
df_test = df_test.drop(['PassengerId'] , axis=1)
df = pd.concat([df_train,df_test])
df.head()

,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [2]:
#只取 int64, float64 兩種數值型欄位, 存於 num_features 中
num_features = []
for dtype, feature in zip(df.dtypes, df.columns):
    if dtype == 'float64' or dtype == 'int64':
        num_features.append(feature)
print(f'{len(num_features)} Numeric Features : {num_features}\n')

5 Numeric Features : ['Pclass', 'Age', 'SibSp', 'Parch', 'Fare']



In [3]:
# 削減文字型欄位, 只剩數值型欄位
df = df[num_features]
train_num = train_Y.shape[0]
df.head()

,Pclass,Age,SibSp,Parch,Fare
0,3,22.0,1,0,7.2500
1,1,38.0,1,0,71.2833
2,3,26.0,0,0,7.9250
3,1,35.0,1,0,53.1000
4,3,35.0,0,0,8.0500


# 作業1
* 試著在補空值區塊, 替換並執行兩種以上填補的缺值, 看看何者比較好?

In [4]:
"""
Your Code Here
"""
estimator = LogisticRegression()

# 空值補 0, 做羅吉斯迴歸
df_m0 = df.fillna(0)
train_X = df_m0[:train_num]
fillna0_cv = cross_val_score(estimator, train_X, train_Y, cv=5).mean()

# 空值補 -1, 做羅吉斯迴歸
df_m1 = df.fillna(-1)
train_X = df_m1[:train_num]
fillna1_cv = cross_val_score(estimator, train_X, train_Y, cv=5).mean()

# 空值補平均, 做羅吉斯迴歸
df_mmean = df.fillna(df.mean())
train_X = df_mmean[:train_num]
fillnamean_cv = cross_val_score(estimator, train_X, train_Y, cv=5).mean()

# 空值補中位數, 做羅吉斯迴歸
df_mmedian = df.fillna(df.median())
train_X = df_mmedian[:train_num]
fillnamedian_cv = cross_val_score(estimator, train_X, train_Y, cv=5).mean()

print('Fill in 0, CV score = %f' % fillna0_cv)
print('Fill in -1, CV score = %f' % fillna1_cv)
print('Fill in mean, CV score = %f' % fillnamean_cv)
print('Fill in median, CV score = %f' % fillnamedian_cv)


Fill in 0, CV score = 0.697154
Fill in -1, CV score = 0.696030
Fill in mean, CV score = 0.698176
Fill in median, CV score = 0.699293


# 作業2
* 使用不同的標準化方式 ( 原值 / 最小最大化 / 標準化 )，搭配羅吉斯迴歸模型，何者效果最好?<br>
Answer: fill in 0 

In [5]:
"""
Your Code Here
"""
# MinMaxScaler
    # fill in 0
df_m0_minmax = MinMaxScaler().fit_transform(df_m0)
train_X = df_m0_minmax[:train_num]
minmax0_cv = cross_val_score(estimator, train_X, train_Y, cv=5).mean()
    # fill in -1
df_m1_minmax = MinMaxScaler().fit_transform(df_m1)
train_X = df_m1_minmax[:train_num]
minmax1_cv = cross_val_score(estimator, train_X, train_Y, cv=5).mean()
    # fill in mean
df_mmean_minmax = MinMaxScaler().fit_transform(df_mmean)
train_X = df_mmean_minmax[:train_num]
minmaxmean_cv = cross_val_score(estimator, train_X, train_Y, cv=5).mean()
    # fill in median
df_mmedian_minmax = MinMaxScaler().fit_transform(df_mmedian)
train_X = df_mmedian_minmax[:train_num]
minmaxmedian_cv = cross_val_score(estimator, train_X, train_Y, cv=5).mean()

# StandardScaler
    # fill in 0
df_m0_standard = StandardScaler().fit_transform(df_m0)
train_X = df_m0_standard[:train_num]
standard0_cv = cross_val_score(estimator, train_X, train_Y, cv=5).mean()
    # fill in -1
df_m1_standard = StandardScaler().fit_transform(df_m1)
train_X = df_m1_standard[:train_num]
standard1_cv = cross_val_score(estimator, train_X, train_Y, cv=5).mean()
    # fill in mean
df_mmean_standard = StandardScaler().fit_transform(df_mmean)
train_X = df_mmean_standard[:train_num]
standardmean_cv = cross_val_score(estimator, train_X, train_Y, cv=5).mean()
    # fill in median
df_mmedian_standard = StandardScaler().fit_transform(df_mmedian)
train_X = df_mmedian_standard[:train_num]
standardmedian_cv = cross_val_score(estimator, train_X, train_Y, cv=5).mean()

print('Fill in 0, CV score = %f' % fillna0_cv)
print('Fill in 0, MinMaxScaler, CV score = %f' % minmax0_cv)
print('Fill in 0, StandardScaler, CV score = %f' % standard0_cv)
print('================================================')
print('Fill in -1, CV score = %f' % fillna1_cv)
print('Fill in -1, MinMaxScaler, CV score = %f' % minmax1_cv)
print('Fill in -1, StandardScaler, CV score = %f' % standard1_cv)
print('================================================')
print('Fill in mean, CV score = %f' % fillnamean_cv)
print('Fill in mean, MinMaxScaler, CV score = %f' % minmaxmean_cv)
print('Fill in mean, StandardScaler, CV score = %f' % standardmean_cv)
print('================================================')
print('Fill in median, CV score = %f' % fillnamedian_cv)
print('Fill in median, MinMaxScaler, CV score = %f' % minmaxmedian_cv)
print('Fill in median, StandardScaler, CV score = %f' % standardmedian_cv)

Fill in 0, CV score = 0.697154
Fill in 0, MinMaxScaler, CV score = 0.701629
Fill in 0, StandardScaler, CV score = 0.698258
Fill in -1, CV score = 0.696030
Fill in -1, MinMaxScaler, CV score = 0.697135
Fill in -1, StandardScaler, CV score = 0.698258
Fill in mean, CV score = 0.698176
Fill in mean, MinMaxScaler, CV score = 0.699350
Fill in mean, StandardScaler, CV score = 0.695941
Fill in median, CV score = 0.699293
Fill in median, MinMaxScaler, CV score = 0.698233
Fill in median, StandardScaler, CV score = 0.697059


In [6]:
dir = {'Original':[fillna0_cv, fillna1_cv, fillnamean_cv, fillnamedian_cv], 'MiniMax':[minmax0_cv, minmax1_cv, minmaxmean_cv, minmaxmedian_cv], \
           'Standard':[standard0_cv, standard1_cv, standardmean_cv, standardmedian_cv]}
df = pd.DataFrame(dir, index=['filling na with 0', 'filling na with -1', 'filling na with mean', 'filling na with median'])
for column in df.columns:
    score = df[column].max()
    for index in df.index:
        if df.loc[index, column] == score:
            method = index
    print('Best VC score = %f in %s by %s' % (score, str(column), str(method)))

Best VC score = 0.699293 in Original by filling na with median
Best VC score = 0.701629 in MiniMax by filling na with 0
Best VC score = 0.698258 in Standard by filling na with -1
